<a href="https://colab.research.google.com/github/timlok123/Learning-the-basic-of-AI-with-Python/blob/main/Ch5_Basic_of_trainning_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ch6 - Possible problem in AI training and Solution 

In [2]:
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.
